In [3]:
def write_db(save_df: DataFrame, table_name: str, mode="overwrite"):
    save_df.write.format("jdbc") \
        .mode(mode) \
        .option("url", "jdbc:mysql://127.0.0.1:3306/cs179g") \
        .option("driver", "com.mysql.cj.jdbc.Driver") \
        .option("dbtable", f"{table_name}") \
        .option("user", "group6") \
        .option("batchsize", "100000") \
        .option("password", "grp6").save()

In [1]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import explode, udf, size, col, concat_ws, to_date, to_timestamp, date_trunc, count, array_contains, month, dayofmonth, hour, explode_outer, split
import plotly.express as px
import plotly.graph_objects as go
import pandas
import numpy as np

spark = SparkSession.builder.appName('practice').config('spark.jars.packages', 'mysql:mysql-connector-java:8.0.30').master(
    "local").getOrCreate()

mdhr_tags = spark.read.format("jdbc").option("url", "jdbc:mysql://localhost:3306/cs179g") \
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "mdhr_tags") \
    .option("user", "Garrett").option("password", "123").load()

mdhr_tags.filter(col('month') != 10).count()

exploded_tags = mdhr_tags.withColumn('tags',split(col('tags'),','))

exploded_tags = exploded_tags.withColumn('tags',explode_outer('tags'))

exploded_tags=exploded_tags.filter(col('tags') != '')

print(exploded_tags.count())
print(exploded_tags.filter(col('tags') != col('tickers')).count())

exploded_tags = exploded_tags.filter(col('tags') != col('tickers'))

exploded_tags.createOrReplaceTempView("exploded_tags")
top_tags=spark.sql("Select tickers, month, day, hour,tags, count(tags) as times from exploded_tags group by tickers,month, day, hour,tags order by times DESC")
top_tags.createOrReplaceTempView("top_tags")



:: loading settings :: url = jar:file:/home/ubuntu/your_venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
mysql#mysql-connector-java added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4c75795e-254a-4752-86ca-b426f529b297;1.0
	confs: [default]
	found mysql#mysql-connector-java;8.0.30 in central
	found com.google.protobuf#protobuf-java;3.19.4 in central
:: resolution report :: resolve 165ms :: artifacts dl 12ms
	:: modules in use:
	com.google.protobuf#protobuf-java;3.19.4 from central in [default]
	mysql#mysql-connector-java;8.0.30 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------

22/11/25 07:31:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/25 07:31:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/11/25 07:31:36 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.


157161
123593


In [2]:
top_tags.show()

+-------+-----+---+----+------------+-----+
|tickers|month|day|hour|        tags|times|
+-------+-----+---+----+------------+-----+
|   META|   11|  3|  12|     ARWEAVE|  375|
|   TEAM|   10| 26|   8|        DEFI|  329|
|   TEAM|   10| 26|   8|       SAFUU|  256|
|   TEAM|   10| 26|   8|       GRAPE|  256|
|   TEAM|   10| 26|   8|  STABLEFUND|  256|
|   TEAM|   10| 26|   8|         EMP|  256|
|   TEAM|   10| 26|   8|        DRIP|  256|
|   TEAM|   10| 26|   8|     CAPTAIN|  255|
|     EA|   10| 26|  16|          FX|  223|
|     EA|   11|  3|  17|          FX|  220|
|   TEAM|   10| 26|  11|        DEFI|  188|
|   TEAM|   10| 26|   8|  JOINTAKEDA|  181|
|   TEAM|   10| 26|   8|BATTLESTAKES|  181|
|   META|   11|  3|  13|     ARWEAVE|  171|
|   TEAM|   10| 26|  11|         EMP|  167|
|   TEAM|   10| 26|  11|  STABLEFUND|  167|
|   TEAM|   10| 26|  11|       GRAPE|  167|
|   TEAM|   10| 26|  11|     CAPTAIN|  167|
|   TEAM|   10| 26|  11|       SAFUU|  167|
|   TEAM|   10| 26|  11|        

In [4]:
write_db(top_tags, 'top_tags')